参考

[Qiita: DatabricksにおけるRAGの実践(前編)](https://qiita.com/taka_yayoi/items/f174599e4721e51e9e1d)

## サマリ
1. PDFからテキスト抽出
2. 文字列チャンク化＋Spark DataFrame化
3. Embeddings計算＋ベクトル保存（ファイル or Delta形式）
4. 簡易MLモデル（例：質問応答用簡易分類器）訓練＋MLflowログ
5. サービング部分は「ローカル用のFastAPIサンプルコードへの移行案」コメント付き

## 1. PDFからテキスト抽出

In [0]:
# 1. 必要なライブラリのインストール
%pip install pdfplumber databricks-langchain


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# 2. Unity Catalog VolumeのPDFファイルパスを直接指定
pdf_volume_path = '/Volumes/my_lab/mydocs_rag/raw_docs/卒論_福永敦史.pdf'

# 3. PDFテキスト抽出関数の定義
import pdfplumber

def extract_pdf_text(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text

# 4. テキスト抽出と表示
pdf_text = extract_pdf_text(pdf_volume_path)
print(pdf_text[:1000])  # 先頭1000文字を表示

令和5年 (2023) 年度 卒業研究論文
行動認識を用いた
不法投棄検知システムの開発
提出者
琉球大学工学部工学科機械工学コース
福永敦史(195158B)
指導教員
上里英輔
令和6年(2024)年2月14日提出目次
第１章 緒言
1.1 研究背景
1.2 本論文の構成
第２章 材料と方法
2.1 MMAction2
2.2 PoseC3D
2.3 データ収集
2.4 骨格データに変換
2.5 モデルのトレーニング
第３章 結果と考察
3.1 骨格データの抽出結果
3.2 学習曲線での評価
3.3 混同行列での評価
第４章 結言
謝辞
参考文献
1図表目次
図 1.1 構内での不法投棄画像（2022年3月撮影)................................3
図 2.1 GCNでの行動認識に使用される骨格の時空間グラフデータ..................5
図 2.2 撮影した動画..........................................................6
図 2.3 抽出する関節部分......................................................7
図 3.1 姿勢推定モデルで動画（左）から抽出した関節点群........................9
図 3.2 データ個数別学習曲線.................................................10
図 3.3 3.2のモデルに30個のテストデータを用いた推論結果（混同行列）.........11
表1 カテゴリごとの再現率と適合率............................................12
2第１章 緒言
1.1 研究背景
昨年度まで、図1.1のように本学構内の複数箇所に家電製品などの不法投棄が数多く見
受けられていた。監視カメラは設置されておらず、不法投棄を取り締まることができてい
なかった。そこで昨年から石澤らにより、Raspberry Pi[15]（以下ラズパイ）と小型カメ
ラと画像認識技術を用いた不法投棄検知システムが開発された[1]。だが、警備員や通行
人も不法投棄者として誤検知されることがあった。現在は不法投棄され

In [0]:
# 5. 文字列チャンク化関数の定義
import math

def chunk_text(text, chunk_size=1000):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

chunks = chunk_text(pdf_text, chunk_size=1000)

# 6. チャンクごとにIDを付与しSpark DataFrame化
chunk_data = [(i, chunk) for i, chunk in enumerate(chunks)]
chunk_df = spark.createDataFrame(chunk_data, ["chunk_id", "text"])

# 7. 先頭数件を表示
display(chunk_df.limit(5))

chunk_id,text
0,令和5年 (2023) 年度 卒業研究論文 行動認識を用いた 不法投棄検知システムの開発 提出者 琉球大学工学部工学科機械工学コース 福永敦史(195158B) 指導教員 上里英輔 令和6年(2024)年2月14日提出目次 第１章 緒言 1.1 研究背景 1.2 本論文の構成 第２章 材料と方法 2.1 MMAction2 2.2 PoseC3D 2.3 データ収集 2.4 骨格データに変換 2.5 モデルのトレーニング 第３章 結果と考察 3.1 骨格データの抽出結果 3.2 学習曲線での評価 3.3 混同行列での評価 第４章 結言 謝辞 参考文献 1図表目次 図 1.1 構内での不法投棄画像（2022年3月撮影)................................3 図 2.1 GCNでの行動認識に使用される骨格の時空間グラフデータ..................5 図 2.2 撮影した動画..........................................................6 図 2.3 抽出する関節部分......................................................7 図 3.1 姿勢推定モデルで動画（左）から抽出した関節点群........................9 図 3.2 データ個数別学習曲線.................................................10 図 3.3 3.2のモデルに30個のテストデータを用いた推論結果（混同行列）.........11 表1 カテゴリごとの再現率と適合率............................................12 2第１章 緒言 1.1 研究背景 昨年度まで、図1.1のように本学構内の複数箇所に家電製品などの不法投棄が数多く見 受けられていた。監視カメラは設置されておらず、不法投棄を取り締まることができてい なかった。そこで昨年から石澤らにより、Raspberry Pi[15]（以下ラズパイ）と小型カメ ラと画像認識技術を用いた不法投棄検知システムが開発された[1]。だが、警備員や通行 人も不法投棄者として誤検知されることがあった。現在は不法投棄され
1,たゴミは撤去され ているが、再発防止のため不法投棄検知精度の向上を目標として本研究を行う。 本研究では、従来の画像認識ではなく行動認識を用いる。行動認識とは、人間や動物の 行動を、画像ではなく動画や音声データを基に、その行動パターンや特徴を特定し、分類 する技術である。前研究では、画像に写った人間が物を持っている条件で検知されていた が、行動認識を用いることにより、物を持っているか否かに関わらず、不法投棄としての 行動をより正確に識別することが期待できる。 図1.1 構内での不法投棄画像(2022年3月撮影) 1.2 本論文の構成 本論文の構成は以下のようになっている。 第2章では、本研究に用いたオープンソースやツールについてと不法投棄検知モデルの作 成方法について示す。第3章では、モデルの訓練結果と考察について述べ、第4章を結言 とする。 3第２章 材料と方法 本研究ではMMAction2[2]のPoseC3D[3]という技術を用いて不法投棄検出モデルを作成し た。以下でその技術の概要と優れている点について述べる。 2.1 MMAction2 MMAction2は、PyTorchを基にした映像認識のためのツールが集まったオープンソースラ イブラリであり、OpenMMLab[4]というオープンソースプラットフォームの一部である。 OpenMMLab は画像分類、物体検出、映像認識などのアルゴリズムや事前学習モデルを提供 している。世界中の開発者がプロジェクトに参加していて、世界の大学、研究機関、企業 で広く使用されている。 2.2 PoseC3D PoseC3D は人間の骨格データを用いた機械学習により行動認識を行う手法の一つで、複 数の大学の研究者が共著した論文[5]により提案された。先に述べたMMAction2にツールと して含まれている。これまでに使われてきた骨格データでの行動認識は、グラフ畳み込み ネットワーク(GCN)[6]を用いたものが多かった。しかし、PoseC3D は GCN と比較して、骨 格のデータをより効果的に捉えることができ、データのノイズや変動に対して頑健で、形 式の異なるデータセットも統合できる汎用性を持つ。 (1) 骨格データでの行動認識に用いられるグラフ畳み込みネットワーク(GCN) GCN ではグラフシーケンスを使って骨格データを表
2,現する。ここで述べるグラフシーケ ンスとは人間の関節部分の座標と、その時間の経過に伴う変化を捉えた図 2.1 のようなグ ラフデータである。しかし、実際にはこのような画像データではなく、骨格の座標データ と時間経過のつながりを表した行列がデータとして使用される。 このデータを使用した学習は、画像処理に用いる畳み込みニューラルネットワーク (CNN)と似た手法で行われる。CNN での画像の畳み込みでは、隣接するピクセルを抽出する ことで、ピクセルの位置関係や距離に基づいて学習を行うことができる。GCN では、時空 間方向と空間方向に対して畳み込みを行う。これにより、CNN と同様に、隣接するノード 間の関係性を学習することができる。つまり、関節の時間変化と関節の位置関係、そのつ ながりを学習することができる。 4図2.1 GCNでの行動認識に使用される骨格の時空間グラフデータ、出典元: [6] (2)PoseC3D GCNとPoseC3Dの大きな違いは、扱うデータの形式である。PoseC3Dでは、人間の関節の 時間経過による変化を表すデータに加えて、関節ごとのデータの信頼度スコアも扱う。こ のデータの形式を3Dヒートマップボリュームといい、動きやポーズの変化だけでなく、骨 格データの信頼性や確からしさも考慮されたデータ表現が可能となる。これにより、信頼 性の低い関節の位置情報があれば、その情報を弱めてモデルが不安定な予測を行うことを 防ぐことができる。 2.3 データ収集 本研究のモデルトレーニングに使用するデータは、手軽に収集でき、夜間でも明瞭な撮 影が可能なスマートフォンを用いている。実際の運用ではラズパイを使用することを想定 している。ただし、モデルのトレーニングにおいて、動画の撮影機材はあまり重要ではな いと考えている。その理由は、動画データを骨格データに変換するためである。2.2で述 べたように、PoseC3Dでのトレーニングでは、画角よりも骨格の時間経過に伴う動きが重 要になるからだ。同様の理由で、撮影時間(昼夜)の違いについても、骨格データが正しく 推定されているのであれば、それほど重要ではない。今回は一般的に不法投棄がされやす いとされる夜間に撮影する。ちなみに、ラズパイで撮影した動画を骨格データに変換する ことは可能である。ラズパイの通常のカメラモジ
3,ュールは夜間の撮影に適さないが、赤外 線カメラを用いれば、ラズパイでも本研究同様の結果が得られると考えられる。 5集めた動画のカテゴリは、物を捨てる動作、歩行、バイクを運転(警備員を想定)の３つ である（図2.2（a）〜（c））。撮影の高さ、場所、角度にバリエーションを持たせてデー タを収集することで、幅広い条件下での情報を取得する。理由は、複数の場所に不法投棄 検知システムを設置するには汎用性を高める必要があると考えるためである。高さは、約 1mから約3mの間で変化させて撮影する。場所は大学構内4箇所で撮影した。角度は、動 作を横と縦、斜めといったように変化させて、3次元空間を意識して撮影する。 (a)物を捨てる (b)バイクに乗る (c)歩く動作 図2.2 撮影した動画 2.4 骨格データの抽出 スマートフォンやラズパイで撮影した動画データの形式では、骨格データは抽出できな い。そのため FFmpeg というメディア変換ツールを使って、mp4 形式に変換する[9]。変換 したデータから PoseC3D のドキュメント[10]を参考に骨格データを抽出し、複数の骨格デ ータを一つの辞書データに収集する。抽出には OpenMMLab の MMDetection[11]と MMPose[12]の機械学習モデルを使用している。MMDetection は物体の検出、MMPose は姿勢 推定を行うライブラリである。本研究では図 2.3 のように、鼻と、左右の目、耳、肩、肘、 手首、臀部、膝、足首の17つの関節部分を抽出する[13]。 6図2.3 抽出する関節部分 2.5 モデルのトレーニング PoseC3Dでのトレーニングのドキュメント[14]を参考に進める。モデルの学習について の設定ファイルに2.4で一つに収集した骨格データと、分類クラス数３(物を捨てる動 作、歩行、バイクを運転)を指定し、モデルをトレーニングする。 7第３章 結果と考察 3.1 骨格データ抽出結果 図 3.1(a)〜(d)に本研究で収集したデータから抽出した骨格データの一部を、３つのカ テゴリごとに示す。右の図が撮影した動画データから骨格データに変換したものを自作プ ログラムにより可視化したものである。 (a) 物を捨てる動作 (b) バイクに乗ってい

In [0]:
# 5. 文字列チャンク化関数の定義
import math

def chunk_text(text, chunk_size=1000):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

chunks = chunk_text(pdf_text, chunk_size=1000)

# 6. チャンクごとにIDを付与しSpark DataFrame化
chunk_data = [(i, chunk) for i, chunk in enumerate(chunks)]
chunk_df = spark.createDataFrame(chunk_data, ["chunk_id", "text"])

# 7. 先頭数件を表示
display(chunk_df.limit(5))

chunk_id,text
0,令和5年 (2023) 年度 卒業研究論文 行動認識を用いた 不法投棄検知システムの開発 提出者 琉球大学工学部工学科機械工学コース 福永敦史(195158B) 指導教員 上里英輔 令和6年(2024)年2月14日提出目次 第１章 緒言 1.1 研究背景 1.2 本論文の構成 第２章 材料と方法 2.1 MMAction2 2.2 PoseC3D 2.3 データ収集 2.4 骨格データに変換 2.5 モデルのトレーニング 第３章 結果と考察 3.1 骨格データの抽出結果 3.2 学習曲線での評価 3.3 混同行列での評価 第４章 結言 謝辞 参考文献 1図表目次 図 1.1 構内での不法投棄画像（2022年3月撮影)................................3 図 2.1 GCNでの行動認識に使用される骨格の時空間グラフデータ..................5 図 2.2 撮影した動画..........................................................6 図 2.3 抽出する関節部分......................................................7 図 3.1 姿勢推定モデルで動画（左）から抽出した関節点群........................9 図 3.2 データ個数別学習曲線.................................................10 図 3.3 3.2のモデルに30個のテストデータを用いた推論結果（混同行列）.........11 表1 カテゴリごとの再現率と適合率............................................12 2第１章 緒言 1.1 研究背景 昨年度まで、図1.1のように本学構内の複数箇所に家電製品などの不法投棄が数多く見 受けられていた。監視カメラは設置されておらず、不法投棄を取り締まることができてい なかった。そこで昨年から石澤らにより、Raspberry Pi[15]（以下ラズパイ）と小型カメ ラと画像認識技術を用いた不法投棄検知システムが開発された[1]。だが、警備員や通行 人も不法投棄者として誤検知されることがあった。現在は不法投棄され
1,たゴミは撤去され ているが、再発防止のため不法投棄検知精度の向上を目標として本研究を行う。 本研究では、従来の画像認識ではなく行動認識を用いる。行動認識とは、人間や動物の 行動を、画像ではなく動画や音声データを基に、その行動パターンや特徴を特定し、分類 する技術である。前研究では、画像に写った人間が物を持っている条件で検知されていた が、行動認識を用いることにより、物を持っているか否かに関わらず、不法投棄としての 行動をより正確に識別することが期待できる。 図1.1 構内での不法投棄画像(2022年3月撮影) 1.2 本論文の構成 本論文の構成は以下のようになっている。 第2章では、本研究に用いたオープンソースやツールについてと不法投棄検知モデルの作 成方法について示す。第3章では、モデルの訓練結果と考察について述べ、第4章を結言 とする。 3第２章 材料と方法 本研究ではMMAction2[2]のPoseC3D[3]という技術を用いて不法投棄検出モデルを作成し た。以下でその技術の概要と優れている点について述べる。 2.1 MMAction2 MMAction2は、PyTorchを基にした映像認識のためのツールが集まったオープンソースラ イブラリであり、OpenMMLab[4]というオープンソースプラットフォームの一部である。 OpenMMLab は画像分類、物体検出、映像認識などのアルゴリズムや事前学習モデルを提供 している。世界中の開発者がプロジェクトに参加していて、世界の大学、研究機関、企業 で広く使用されている。 2.2 PoseC3D PoseC3D は人間の骨格データを用いた機械学習により行動認識を行う手法の一つで、複 数の大学の研究者が共著した論文[5]により提案された。先に述べたMMAction2にツールと して含まれている。これまでに使われてきた骨格データでの行動認識は、グラフ畳み込み ネットワーク(GCN)[6]を用いたものが多かった。しかし、PoseC3D は GCN と比較して、骨 格のデータをより効果的に捉えることができ、データのノイズや変動に対して頑健で、形 式の異なるデータセットも統合できる汎用性を持つ。 (1) 骨格データでの行動認識に用いられるグラフ畳み込みネットワーク(GCN) GCN ではグラフシーケンスを使って骨格データを表
2,現する。ここで述べるグラフシーケ ンスとは人間の関節部分の座標と、その時間の経過に伴う変化を捉えた図 2.1 のようなグ ラフデータである。しかし、実際にはこのような画像データではなく、骨格の座標データ と時間経過のつながりを表した行列がデータとして使用される。 このデータを使用した学習は、画像処理に用いる畳み込みニューラルネットワーク (CNN)と似た手法で行われる。CNN での画像の畳み込みでは、隣接するピクセルを抽出する ことで、ピクセルの位置関係や距離に基づいて学習を行うことができる。GCN では、時空 間方向と空間方向に対して畳み込みを行う。これにより、CNN と同様に、隣接するノード 間の関係性を学習することができる。つまり、関節の時間変化と関節の位置関係、そのつ ながりを学習することができる。 4図2.1 GCNでの行動認識に使用される骨格の時空間グラフデータ、出典元: [6] (2)PoseC3D GCNとPoseC3Dの大きな違いは、扱うデータの形式である。PoseC3Dでは、人間の関節の 時間経過による変化を表すデータに加えて、関節ごとのデータの信頼度スコアも扱う。こ のデータの形式を3Dヒートマップボリュームといい、動きやポーズの変化だけでなく、骨 格データの信頼性や確からしさも考慮されたデータ表現が可能となる。これにより、信頼 性の低い関節の位置情報があれば、その情報を弱めてモデルが不安定な予測を行うことを 防ぐことができる。 2.3 データ収集 本研究のモデルトレーニングに使用するデータは、手軽に収集でき、夜間でも明瞭な撮 影が可能なスマートフォンを用いている。実際の運用ではラズパイを使用することを想定 している。ただし、モデルのトレーニングにおいて、動画の撮影機材はあまり重要ではな いと考えている。その理由は、動画データを骨格データに変換するためである。2.2で述 べたように、PoseC3Dでのトレーニングでは、画角よりも骨格の時間経過に伴う動きが重 要になるからだ。同様の理由で、撮影時間(昼夜)の違いについても、骨格データが正しく 推定されているのであれば、それほど重要ではない。今回は一般的に不法投棄がされやす いとされる夜間に撮影する。ちなみに、ラズパイで撮影した動画を骨格データに変換する ことは可能である。ラズパイの通常のカメラモジ
3,ュールは夜間の撮影に適さないが、赤外 線カメラを用いれば、ラズパイでも本研究同様の結果が得られると考えられる。 5集めた動画のカテゴリは、物を捨てる動作、歩行、バイクを運転(警備員を想定)の３つ である（図2.2（a）〜（c））。撮影の高さ、場所、角度にバリエーションを持たせてデー タを収集することで、幅広い条件下での情報を取得する。理由は、複数の場所に不法投棄 検知システムを設置するには汎用性を高める必要があると考えるためである。高さは、約 1mから約3mの間で変化させて撮影する。場所は大学構内4箇所で撮影した。角度は、動 作を横と縦、斜めといったように変化させて、3次元空間を意識して撮影する。 (a)物を捨てる (b)バイクに乗る (c)歩く動作 図2.2 撮影した動画 2.4 骨格データの抽出 スマートフォンやラズパイで撮影した動画データの形式では、骨格データは抽出できな い。そのため FFmpeg というメディア変換ツールを使って、mp4 形式に変換する[9]。変換 したデータから PoseC3D のドキュメント[10]を参考に骨格データを抽出し、複数の骨格デ ータを一つの辞書データに収集する。抽出には OpenMMLab の MMDetection[11]と MMPose[12]の機械学習モデルを使用している。MMDetection は物体の検出、MMPose は姿勢 推定を行うライブラリである。本研究では図 2.3 のように、鼻と、左右の目、耳、肩、肘、 手首、臀部、膝、足首の17つの関節部分を抽出する[13]。 6図2.3 抽出する関節部分 2.5 モデルのトレーニング PoseC3Dでのトレーニングのドキュメント[14]を参考に進める。モデルの学習について の設定ファイルに2.4で一つに収集した骨格データと、分類クラス数３(物を捨てる動 作、歩行、バイクを運転)を指定し、モデルをトレーニングする。 7第３章 結果と考察 3.1 骨格データ抽出結果 図 3.1(a)〜(d)に本研究で収集したデータから抽出した骨格データの一部を、３つのカ テゴリごとに示す。右の図が撮影した動画データから骨格データに変換したものを自作プ ログラムにより可視化したものである。 (a) 物を捨てる動作 (b) バイクに乗ってい

In [0]:
# --- Databricks Foundation Model APIでembedding計算 ---
from databricks_langchain import DatabricksEmbeddings
import pandas as pd

embeddings_model = DatabricksEmbeddings(endpoint="databricks-bge-large-en")

# pandas UDFでembeddingを計算
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import ArrayType, FloatType

@pandas_udf(ArrayType(FloatType()))
def get_embedding_udf(texts: pd.Series) -> pd.Series:
    return pd.Series(embeddings_model.embed_documents(texts.tolist()))

chunk_df = chunk_df.withColumn("embedding", get_embedding_udf(chunk_df["text"]))

# Deltaテーブルとして保存
chunk_embeddings_table = "my_lab.mydocs_rag.chunk_embeddings"
chunk_df.write.mode("overwrite").saveAsTable(chunk_embeddings_table)

# 保存後の確認表示
display(spark.table(chunk_embeddings_table).limit(5))

chunk_id text embedding 0 令和5年 (2023) 年度 卒業研究論文
行動認識を用いた
不法投棄検知システムの開発
提出者
琉球大学工学部工学科機械工学コース
福永敦史(195158B)
指導教員
上里英輔
令和6年(2024)年2月14日提出目次
第１章 緒言
1.1 研究背景
1.2 本論文の構成
第２章 材料と方法
2.1 MMAction2
2.2 PoseC3D
2.3 データ収集
2.4 骨格データに変換
2.5 モデルのトレーニング
第３章 結果と考察
3.1 骨格データの抽出結果
3.2 学習曲線での評価
3.3 混同行列での評価
第４章 結言
謝辞
参考文献
1図表目次
図 1.1 構内での不法投棄画像（2022年3月撮影)................................3
図 2.1 GCNでの行動認識に使用される骨格の時空間グラフデータ..................5
図 2.2 撮影した動画..........................................................6
図 2.3 抽出する関節部分......................................................7
図 3.1 姿勢推定モデルで動画（左）から抽出した関節点群........................9
図 3.2 データ個数別学習曲線.................................................10
図 3.3 3.2のモデルに30個のテストデータを用いた推論結果（混同行列）.........11
表1 カテゴリごとの再現率と適合率............................................12
2第１章 緒言
1.1 研究背景
昨年度まで、図1.1のように本学構内の複数箇所に家電製品などの不法投棄が数多く見
受けられていた。監視カメラは設置されておらず、不法投棄を取り締まることができてい
なかった。そこで昨年から石澤らにより、Raspberry Pi[15]（以下ラズパイ）と小型カメ
ラと画像認識技術を用いた不法投棄検知システムが開発された[1]。だが、警備員や通行
人も不法投棄者として誤検知されることがあった。現在は不法投棄され List(-0.0072898865, -0.014579773, 0.029022217, 0.0115356445, -0.015464783, -0.012939453, 0.039123535, 0.01373291, 0.01876831, 0.047332764, 0.013870239, 0.023132324, 0.04537964, -0.019866943, -0.049224854, -0.051605225, 0.0076904297, 0.009490967, -0.05026245, 0.02305603, 0.03842163, 0.03857422, -0.09613037, -0.026809692, -0.0129470825, 0.02658081, 0.047302246, 0.0034599304, 0.044433594, 0.043914795, 0.0054779053, -0.012001038, -0.0104904175, -0.043792725, -0.0028266907, 0.0054244995, 0.05407715, -0.04675293, 0.009407043, -0.035614014, 0.017425537, -0.026565552, 0.057250977, -0.025772095, -0.037597656, -0.028244019, 0.0022411346, -0.0022068024, -0.017211914, -0.03375244, 0.0052223206, 0.009025574, 0.06463623, -0.023742676, 0.01914978, -0.0030956268, -0.0053215027, -0.0021381378, -0.023773193, 0.03060913, 0.025741577, 0.027999878, 0.04171753, -0.06390381, -0.014549255, 0.006126404, 0.02003479, 0.018539429, 0.011138916, -0.019729614, -0.030197144, 0.011940002, -0.038208008, 0.014511108, -0.059509277, -0.011497498, 0.015670776, 0.010406494, -0.043518066, 0.05633545, -0.014045715, 0.010063171, 0.020339966, -0.034698486, 0.015731812, -0.051574707, 0.028274536, 0.001074791, -0.010757446, 0.0013723373, -0.013519287, 0.04296875, -0.054595947, -0.020401001, -0.002571106, 0.011581421, -0.0126953125, 0.0066604614, -0.012245178, -0.013290405, 0.021636963, 0.058258057, -0.024047852, 0.053222656, -0.053466797, 7.009506E-4, -0.021072388, 0.020629883, 0.0072288513, -0.03591919, 0.0010156631, 4.9972534E-4, 0.016723633, -0.0019798279, -0.0018844604, 0.059173584, 4.0268898E-4, 0.0061836243, -0.045928955, -0.020355225, 0.01777649, -0.011161804, 0.014457703, 0.0031147003, 0.023605347, -0.022476196, 0.007411957, 0.069885254, -0.020614624, 0.016525269, 0.009849548, 0.0041770935, -0.015045166, -0.02305603, -0.0345459, 0.044311523, -0.024124146, -0.005996704, 0.029510498, -0.05618286, 0.0030136108, 0.008026123, -0.016113281, 0.0793457, -0.030258179, 0.024536133, -0.025756836, 0.010345459, -0.03250122, 0.007358551, -0.03829956, -0.004085541, 0.005455017, 0.0064964294, -0.0049438477, 0.023529053, -0.0096588135, 0.040374756, -0.01776123, 0.0055656433, 0.014381409, 0.01436615, -0.020965576, 0.036224365, 0.008430481, 0.04385376, -0.032043457, 0.0051651, -0.005039215, -0.030792236, 0.020324707, 0.0074310303, -0.008560181, -0.0038089752, 0.0368042, 0.046905518, -0.0088272095, 0.001244545, 0.018753052, 0.047821045, -0.012649536, -0.0060272217, 0.034851074, 0.025146484, 0.016143799, 0.024169922, -0.022644043, -0.04852295, -0.0039711, -0.016540527, -0.0061416626, 0.059814453, -5.631447E-4, 0.021896362, 0.021697998, 0.002588272, -0.06713867, -0.0018672943, 0.028305054, -0.052734375, -0.037963867, 0.03894043, 0.0011024475, 0.028808594, -0.027496338, -0.011116028, 0.03753662, 0.070495605, -0.015159607, 0.040405273, 0.022644043, 0.008331299, -0.0067443848, 0.023483276, 0.045013428, -0.006252289, -0.038085938, 0.0317688, -0.031555176, 0.041809082, 0.017944336, 

## ベクター検索エンドポイントとベクター検索インデックスを作成

手順の参考：[Azure｜ベクター検索インデックスを作成してクエリを実行する方法](https://learn.microsoft.com/ja-jp/azure/databricks/generative-ai/create-query-vector-search)

ベクター検索を使うためには、主に次の3つの要素が必要です：

1. ベクター検索エンドポイント（検索を提供するAPI窓口）
2. ベクター検索インデックス（検索対象のデータ本体）
3. 埋め込み（Embedding）モデル（テキストを数値ベクトルに変換）

### ベクター検索エンドポイント

参考：[Microsoft｜モザイク AI ベクター検索を設定する方法](https://learn.microsoft.com/ja-jp/azure/databricks/generative-ai/vector-search#how-to-set-up-mosaic-ai-vector-search)

いわば「検索サーバー」や「検索APIの入り口」。

インデックスをホストしている実体です。

REST API や SDK を通じて、検索クエリを送ったり、インデックスを更新したりできます。

例：「この文章に似たドキュメントを探して」と問い合わせると、類似データを返す役割。

エンドポイントにはは2種類ある。

| 種類                  | 特徴                       |
| ------------------- | ------------------------ |
| **ストレージ最適化エンドポイント** | インデックス削除時に自動スケールダウン（節約型） |
| **標準エンドポイント**       | 自動スケールダウンなし       |


### ベクター検索インデックスを作成

検索対象となる 「埋め込みベクトルを格納したデータ構造」 です。
SQLでいうテーブルに似ていますが、「類似検索（Approximate Nearest Neighbor, ANN）」に最適化されています。

- ソースは Deltaテーブル（＝元データ）
- DatabricksがそのDeltaをもとに、検索用のインデックスを構築します
- Unity Catalogで管理され、権限やデータガバナンスも統合されています

**目的**

「クエリ（質問）に似たベクトルをすばやく探す」こと。
つまり、自然言語やドキュメントの**意味的な検索（semantic search）**が可能になります。

In [0]:
# from databricks.sdk import WorkspaceClient

# w = WorkspaceClient()
# openai_client = w.serving_endpoints.get_open_ai_client()

# # 修正: 正しいエンドポイント名に変更
# response = openai_client.embeddings.create(
#   model="databricks-bge-large-en",
#   input="what is databricks"
# )

# import numpy as np

# def is_normalized(vector: list[float], tol=1e-3) -> bool:
#     magnitude = np.linalg.norm(vector)
#     return abs(magnitude - 1) < tol

# # エンベディングの動作確認
# embedding_vector = response.data[0].embedding
# print(is_normalized(embedding_vector))
# print(f"response:{response}")

True
response:CreateEmbeddingResponse(data=[Embedding(embedding=[-0.01727294921875, -0.0100250244140625, -0.020599365234375, 0.0020465850830078125, 0.0094146728515625, -0.01099395751953125, 0.0224456787109375, -0.008453369140625, 0.01218414306640625, 0.02899169921875, -0.040435791015625, -0.0087738037109375, 0.054840087890625, -0.034393310546875, -0.01245880126953125, -0.00910186767578125, -0.03192138671875, 0.0178680419921875, -0.043670654296875, 0.0152740478515625, 0.0293731689453125, 0.04296875, -0.044708251953125, -0.0164642333984375, -0.0177001953125, 0.047210693359375, -0.0005440711975097656, 0.0272064208984375, 0.09112548828125, 0.0565185546875, -0.03314208984375, -0.0229034423828125, 0.0224761962890625, -0.036285400390625, -0.005031585693359375, 0.01044464111328125, 0.00974273681640625, -0.0157012939453125, -0.0271453857421875, -0.01544189453125, 0.0135650634765625, 0.032257080078125, 0.026885986328125, -0.045257568359375, -0.045013427734375, -0.0192718505859375, -0.015625, -0.